In [1]:
pip install ase

In [ ]:
"""
Generates conventional unit cells for both gamma-TiAl (L1_0) and alpha2-Ti3Al (D0_19)
phases using the Atomic Simulation Environment (ASE) package "only for comparison actual generating code is in the cell below".

Loops through all replication factor combinations from (1,1) to (10,10)
for both lattices to find the combinations with the lowest
Primarily used ofr X-dimension and Y-dimension mismatch.

"""

import sys
import math
from ase import Atoms
from ase.io import write
from ase.build import cut


def create_gamma_tial_unit_cell():

    a = 4.02
    c_over_a = 1.02
    c = a * c_over_a

    cell = [a, a, c]
    symbols = ['Ti', 'Ti', 'Al', 'Al']

    scaled_positions = [
        [0.0, 0.0, 0.0],  # Ti 1
        [0.5, 0.5, 0.0],  # Ti 2
        [0.0, 0.5, 0.5],  # Al 1
        [0.5, 0.0, 0.5]   # Al 2
    ]

    gamma_tial = Atoms(
        symbols=symbols,
        scaled_positions=scaled_positions,
        cell=cell,
        pbc=True  # Periodic boundary condition
    )

    return gamma_tial


def create_alpha2_ti3al_unit_cell():

    
    a = 5.75
    c_over_a = 0.806
    c = a * c_over_a 


    cell = [
        [a, 0, 0],
        [-a * 0.5, a * math.sqrt(3) / 2, 0],
        [0, 0, c]
    ]

    # --- Atomic symbols (2 Al, 6 Ti) ---
    symbols = ['Al'] * 2 + ['Ti'] * 6

    # --- Scaled atomic positions for D0_19 structure ---
    # Wyckoff positions: 2c for Al, 6h for Ti (with x=5/6)
    scaled_positions = [
        # Al atoms (2c)
        [1/3, 2/3, 0.25],
        [2/3, 1/3, 0.75],

        # Ti atoms (6h) with x = 5/6
        [5/6, 2/3, 0.25],
        [1/3, 1/6, 0.25],
        [5/6, 1/6, 0.25],
        [1/6, 1/3, 0.75],
        [2/3, 5/6, 0.75],
        [1/6, 5/6, 0.75]
    ]

    # Create the Atoms object
    alpha2_ti3al = Atoms(
        symbols=symbols,
        scaled_positions=scaled_positions,
        cell=cell,
        pbc=True
    )

    return alpha2_ti3al


def preview_file_content(filename):
    try:
        print(f"\n--- File Content Preview: {filename} (first 15 lines) ---")
        with open(filename, 'r') as f:
            for i in range(15):
                line = f.readline()
                if not line:
                    break
                print(line.strip())
        print("---------------------------------------------")
    except Exception as e:
        print(f"Could not read file preview for {filename}: {e}")


def get_oriented_unit_cell_dims():

    print("--- Calculating Oriented Unit Cell Dimensions ---")

    g_a, g_b, a_a, a_b = None, None, None, None

    #Gamma-TiAl
    try:
        gamma_base_cell = create_gamma_tial_unit_cell()
        gamma_oriented_cell = cut(
            gamma_base_cell,
            a=[1, -1, 0],  
            b=[1, 1, -2],  
            c=[1, 1, 1] 
        )

        gamma_params = gamma_oriented_cell.get_cell().cellpar()
        g_a, g_b, _, _, _, g_gamma_deg = gamma_params
        print(f"Gamma (L1_0) Oriented Unit Cell: a={g_a:.4f} A, b={g_b:.4f} A, angle={g_gamma_deg:.2f} deg")
        if abs(g_gamma_deg - 90.0) > 0.1:
            print("Warning: Gamma unit cell XY plane is not orthogonal (90 deg).")

    except Exception as e:
        print(f"Error calculating gamma unit cell dims: {e}")
        return None, None, None, None

    #Alpha2-Ti3Al
    try:
        alpha2_base_cell = create_alpha2_ti3al_unit_cell()
        alpha2_oriented_cell = cut(
            alpha2_base_cell,
            a=[1, -1, 0], 
            b=[1, 1, 0],   
            c=[0, 0, 1]   
        )

        alpha2_params = alpha2_oriented_cell.get_cell().cellpar()
        a_a, a_b, _, _, _, a_gamma_deg = alpha2_params
        print(f"Alpha2 (D0_19) Oriented Unit Cell: a={a_a:.4f} A, b={a_b:.4f} A, angle={a_gamma_deg:.2f} deg")
        if abs(a_gamma_deg - 90.0) > 0.1:
            print("Warning: Alpha2 unit cell XY plane is not orthogonal (90 deg).")

    except Exception as e:
        print(f"Error calculating alpha2 unit cell dims: {e}")
        return None, None, None, None

    return g_a, g_b, a_a, a_b


def calculate_all_combinations(g_a, g_b, a_a, a_b, max_rep=10):
    if g_a is None or g_b is None or a_a is None or a_b is None:
        print("Cannot calculate combinations, unit cell dimensions are invalid.")
        return

    print(f"\n--- Searching all (1x1) to ({max_rep}x{max_rep}) combinations... ---")
    results = []

    for g_x in range(1, max_rep + 1):
        for g_y in range(1, max_rep + 1):
            total_g_x_len = g_a * g_x
            total_g_y_len = g_b * g_y
            gamma_rep = (g_x, g_y)

            for a_x in range(1, max_rep + 1):
                for a_y in range(1, max_rep + 1):
                    total_a_x_len = a_a * a_x
                    total_a_y_len = a_b * a_y
                    alpha2_rep = (a_x, a_y)

                    # Calculates mismatch for X and Y dimensions 
                    mismatch_x = 100 * abs(total_g_x_len - total_a_x_len) / min(total_g_x_len, total_a_x_len)
                    mismatch_y = 100 * abs(total_g_y_len - total_a_y_len) / min(total_g_y_len, total_a_y_len)

                    
                    total_mismatch = mismatch_x + mismatch_y

                    # Store results
                    results.append((total_mismatch, mismatch_x, mismatch_y, gamma_rep, alpha2_rep, total_g_x_len, total_a_x_len, total_g_y_len, total_a_y_len))


    results.sort(key=lambda x: x[0])

    print("\n--- Best X/Y Dimension Matches (sorted by Total M = M_x + M_y) ---")
    print("Total M (%) | M_x (%) | M_y (%) | Gamma (x,y) | Alpha2 (x,y) | G_X_len | A_X_len | G_Y_len | A_Y_len")
    print("-" * 105)
    for i in range(min(25, len(results))):
        m, m_x, m_y, g_rep, a_rep, g_x_len, a_x_len, g_y_len, a_y_len = results[i]
        print(f"{m:11.3f} | {m_x:7.3f} | {m_y:7.3f} | {str(g_rep):<11} | {str(a_rep):<12} | {g_x_len:7.2f} | {a_x_len:7.2f} | {g_y_len:7.2f} | {a_y_len:7.2f}")


def generate_files_for_specific_combo(gamma_replication, alpha2_replication):

    print(f"\n--- Generating Files for Gamma={gamma_replication}, Alpha2={alpha2_replication} ---")

    
    total_area_gamma = 0.0
    total_area_alpha2 = 0.0
    area_gamma_unit = 0.0
    area_alpha2_unit = 0.0


    output_filename_gamma = f"gamma_tial_oriented_{gamma_replication[0]}x{gamma_replication[1]}x{gamma_replication[2]}.data"

    print(f"\n--- Generating {output_filename_gamma} ---")
    print("Orientation: X=[1-10], Y=[11-2], Z=[111]")

    try:
        
        gamma_base_cell = create_gamma_tial_unit_cell()

        print("\nOriginal cell vectors (default [100], [010], [001]):")
        print(gamma_base_cell.get_cell())

    
        gamma_oriented_cell = cut(
            gamma_base_cell,
            a=[1, -1, 0], 
            b=[1, 1, -2],  
            c=[1, 1, 1]    
        )

        print(f"\nOriented unit cell vectors (before replication):")
        print(gamma_oriented_cell.get_cell())
        print(f"Atoms in unit cell: {len(gamma_oriented_cell)}")

        
        gamma_params = gamma_oriented_cell.get_cell().cellpar()
        g_a, g_b, _, _, _, g_gamma_deg = gamma_params
        area_gamma_unit = g_a * g_b * math.sin(math.radians(g_gamma_deg))
        print(f"Oriented unit cell basal area (a*b*sin(gamma)): {area_gamma_unit:.4f} A^2")

        
        print(f"\nReplicating to {gamma_replication} supercell...")
        gamma_supercell = gamma_oriented_cell.repeat(gamma_replication)


        total_area_gamma = area_gamma_unit * gamma_replication[0] * gamma_replication[1]
        print(f"Total supercell XY area: {total_area_gamma:.4f} A^2")

        print(f"\nFinal supercell vectors for {output_filename_gamma} (Angstroms):")
        print(gamma_supercell.get_cell())
        print(f"Total Atoms in supercell: {len(gamma_supercell)}")

        # Write to a LAMMPS data file
        write(output_filename_gamma, gamma_supercell, format="lammps-data")
        print(f"\nSuccessfully wrote LAMMPS data file to: {output_filename_gamma}")
        preview_file_content(output_filename_gamma)

    except Exception as e:
        print(f"\nAn error occurred during creation or writing for {output_filename_gamma}: {e}")


    #Generate D0_19 (alpha2-Ti3Al) with paper orientation 

    output_filename_alpha2 = f"alpha2_ti3al_oriented_{alpha2_replication[0]}x{alpha2_replication[1]}x{alpha2_replication[2]}.data"

    print(f"\n--- Generating {output_filename_alpha2} ---")
    print("Orientation: X=[1-10], Y=[110], Z=[001]")

    try:
    
        alpha2_base_cell = create_alpha2_ti3al_unit_cell()

        print("\nOriginal cell vectors (default hexagonal):")
        print(alpha2_base_cell.get_cell())

    
        alpha2_oriented_cell = cut(
            alpha2_base_cell,
            a=[1, -1, 0],  
            b=[1, 1, 0],   
            c=[0, 0, 1]    
        )

        print(f"\nOriented unit cell vectors (before replication):")
        print(alpha2_oriented_cell.get_cell())
        print(f"Atoms in unit cell: {len(alpha2_oriented_cell)}")

        alpha2_params = alpha2_oriented_cell.get_cell().cellpar()
        a_a, a_b, _, _, _, a_gamma_deg = alpha2_params
        area_alpha2_unit = a_a * a_b * math.sin(math.radians(a_gamma_deg))
        print(f"Oriented unit cell basal area (a*b*sin(gamma)): {area_alpha2_unit:.4f} A^2")

        print(f"\nReplicating to {alpha2_replication} supercell...")
        alpha2_supercell = alpha2_oriented_cell.repeat(alpha2_replication)

        total_area_alpha2 = area_alpha2_unit * alpha2_replication[0] * alpha2_replication[1]
        print(f"Total supercell XY area: {total_area_alpha2:.4f} A^2")

        print(f"\nFinal supercell vectors for {output_filename_alpha2} (Angstroms):")
        print(alpha2_supercell.get_cell())
        print(f"Total Atoms in supercell: {len(alpha2_supercell)}")

        write(output_filename_alpha2, alpha2_supercell, format="lammps-data")
        print(f"\nSuccessfully wrote LAMMPS data file to: {output_filename_alpha2}")
        preview_file_content(output_filename_alpha2)

    except Exception as e:
        print(f"\nAn error occurred during creation or writing for {output_filename_alpha2}: {e}")

    # --- 4. Compare Final Areas ---
    print("\n--- Final Area Comparison ---")
    if total_area_gamma > 0 and total_area_alpha2 > 0:
        print(f"Gamma supercell area ({gamma_replication[0]}x{gamma_replication[1]}): {total_area_gamma:.4f} A^2")
        print(f"Alpha2 supercell area ({alpha2_replication[0]}x{alpha2_replication[1]}): {total_area_alpha2:.4f} A^2")

        mismatch_percent = 100 * abs(total_area_gamma - total_area_alpha2) / min(total_area_gamma, total_area_alpha2)

        print(f"\nArea Mismatch: {mismatch_percent:.2f} %")
    else:
        print("Could not compare areas. One or both structures failed to generate.")


def main():

    try:
        # Check for ASE import success once
        from ase import Atoms
        from ase.io import write
        from ase.build import cut
    except ImportError:
        print("Error: The 'ase' library is required.")
        print("Please install it using: pip install ase")
        sys.exit(1)

    #Get Unit Cell Dimensions 
    g_a, g_b, a_a, a_b = get_oriented_unit_cell_dims()

    # Calculate All Combinations
   
    calculate_all_combinations(g_a, g_b, a_a, a_b, max_rep=15) ##############################################################################
                                                               ###### Chage this   
if __name__ == "__main__":
    main()



--- Calculating Oriented Unit Cell Dimensions ---
Gamma (L1_0) Oriented Unit Cell: a=5.6851 A, b=9.9787 A, angle=90.00 deg
Alpha2 (D0_19) Oriented Unit Cell: a=9.9593 A, b=5.7500 A, angle=90.00 deg

--- Searching all (1x1) to (15x15) combinations... ---

--- Best X/Y Dimension Matches (sorted by Total M = M_x + M_y) ---
Total M (%) | M_x (%) | M_y (%) | Gamma (x,y) | Alpha2 (x,y) | G_X_len | A_X_len | G_Y_len | A_Y_len
---------------------------------------------------------------------------------------------------------
      0.944 |   0.104 |   0.840 | (7, 4)      | (4, 7)       |   39.80 |   39.84 |   39.91 |   40.25
      0.944 |   0.104 |   0.840 | (7, 8)      | (4, 14)      |   39.80 |   39.84 |   79.83 |   80.50
      0.944 |   0.104 |   0.840 | (14, 4)     | (8, 7)       |   79.59 |   79.67 |   39.91 |   40.25
      0.944 |   0.104 |   0.840 | (14, 8)     | (8, 14)      |   79.59 |   79.67 |   79.83 |   80.50
      1.336 |   0.104 |   1.233 | (7, 7)      | (4, 12)      |   39

In [ ]:
#!/usr/bin/env python3

"""
Generates conventional unit cells for both gamma-TiAl (L1_0) and alpha2-Ti3Al (D0_19)
phases using the Atomic Simulation Environment (ASE) package.

This modified script reorients the final unit cells to match the
orientations specified in the paper text:
- L1_0:     X=[1-10], Y=[11-2], Z=[111]
- D0_19:    X=[1-10], Y=[110],  Z=[001] (using [1-100], [11-20], [0001] Miller-Bravais)

The lattice parameters are kept from the original script (Kim et al., 2016).
P.S. I only used this on Google Colab
"""

import sys
import math
from ase import Atoms
from ase.io import write
from ase.build import cut

def create_gamma_tial_unit_cell():

    # Lattice parameters 
    a = 4.02
    c_over_a = 1.02
    c = a * c_over_a

    # Define the L1_0 orthorhombic cell
    cell = [a, a, c]

    symbols = ['Ti', 'Ti', 'Al', 'Al']

    # --- Scaled atomic positions ---
    scaled_positions = [
        [0.0, 0.0, 0.0],  # Ti 1
        [0.5, 0.5, 0.0],  # Ti 2
        [0.0, 0.5, 0.5],  # Al 1
        [0.5, 0.0, 0.5]   # Al 2
    ]

    # Create the Atoms object
    gamma_tial = Atoms(
        symbols=symbols,
        scaled_positions=scaled_positions,
        cell=cell,
        pbc=True  # Periodic boundary conditions
    )

    return gamma_tial


def create_alpha2_ti3al_unit_cell():
    """
    Creates and returns an ASE Atoms object for the D0_19 Ti3Al
    conventional unit cell, which contains 8 atoms (6 Ti, 2 Al).
    (Lattice parameters from Kim et al., 2016)
    """
    # Lattice parameters from Kim et al. (2016)
    a = 5.75
    c_over_a = 0.806
    c = a * c_over_a  # 4.6345

    # Define the D0_19 hexagonal cell (a, a, c, 90, 90, 120)
    cell = [
        [a, 0, 0],
        [-a * 0.5, a * math.sqrt(3) / 2, 0],
        [0, 0, c]
    ]

    # Atomic symbols (2 Al, 6 Ti) 
    symbols = ['Al'] * 2 + ['Ti'] * 6

    # --- Scaled atomic positions for D0_19 structure ---
    # Wyckoff positions: 2c for Al, 6h for Ti (with x=5/6)
    scaled_positions = [
        # Al atoms (2c)
        [1/3, 2/3, 0.25],
        [2/3, 1/3, 0.75],

        # Ti atoms (6h) with x = 5/6
        [5/6, 2/3, 0.25],
        [1/3, 1/6, 0.25],
        [5/6, 1/6, 0.25],
        [1/6, 1/3, 0.75],
        [2/3, 5/6, 0.75],
        [1/6, 5/6, 0.75]
    ]

    # Create the Atoms object
    alpha2_ti3al = Atoms(
        symbols=symbols,
        scaled_positions=scaled_positions,
        cell=cell,
        pbc=True
    )

    return alpha2_ti3al


def preview_file_content(filename):
    """Prints a preview of the generated file."""
    try:
        print(f"\n--- File Content Preview: {filename} (first 15 lines) ---")
        with open(filename, 'r') as f:
            for i in range(15):
                line = f.readline()
                if not line:
                    break
                print(line.strip())
        print("---------------------------------------------")
    except Exception as e:
        print(f"Could not read file preview for {filename}: {e}")


def main():


    # Generate L1_0 (gamma-TiAl) with orientation 
    output_filename_gamma = "gamma_tial_oriented_4x4x4.data"
    print(f"\n--- Generating {output_filename_gamma} ---")
    print("Orientation: X=[1-10], Y=[11-2], Z=[111]")

    try:
        # Create the base unit cell
        gamma_base_cell = create_gamma_tial_unit_cell()

        print("\nOriginal cell vectors (default [100], [010], [001]):")
        print(gamma_base_cell.get_cell())

        gamma_oriented_cell = cut(
            gamma_base_cell,
            a=[1, -1, 0],  
            b=[1, 1, -2],  
            c=[1, 1, 1]   
        )

        print(f"\nOriented unit cell vectors (before replication):")
        print(gamma_oriented_cell.get_cell())
        print(f"Atoms in unit cell: {len(gamma_oriented_cell)}")
        print(f"Oriented cell PBC: {gamma_oriented_cell.get_pbc()}") # DEBUG line

        gamma_supercell = gamma_oriented_cell.repeat((7, 4, 4)) #####################################################################
        ########################################################## use this for repeat

        print(f"\nFinal supercell vectors for {output_filename_gamma} (Angstroms):")
        print(gamma_supercell.get_cell())
        print(f"Total Atoms in supercell: {len(gamma_supercell)}")

        # Write to a LAMMPS data file
        write(output_filename_gamma, gamma_supercell, format="lammps-data")
        print(f"\nSuccessfully wrote LAMMPS data file to: {output_filename_gamma}")
        preview_file_content(output_filename_gamma)

    except Exception as e:
        print(f"\nAn error occurred during creation or writing for {output_filename_gamma}: {e}")


    # 2. Generate D0_19 (alpha2-Ti3Al) with paper orientation
    output_filename_alpha2 = "alpha2_ti3al_oriented_4x4x4.data"
    print(f"\n--- Generating {output_filename_alpha2} ---")
    print("Orientation: X=[1-10], Y=[110], Z=[001]")

    try:
    
        alpha2_base_cell = create_alpha2_ti3al_unit_cell()

        print("\nOriginal cell vectors (default hexagonal):")
        print(alpha2_base_cell.get_cell())


        alpha2_oriented_cell = cut(
            alpha2_base_cell,
            a=[1, -1, 0],  
            b=[1, 1, 0],  
            c=[0, 0, 1]   
        )

        print(f"\nOriented unit cell vectors (before replication):")
        print(alpha2_oriented_cell.get_cell())
        print(f"Atoms in unit cell: {len(alpha2_oriented_cell)}")
        print(f"Oriented cell PBC: {alpha2_oriented_cell.get_pbc()}") # DEBUG line

        alpha2_supercell = alpha2_oriented_cell.repeat((4, 7, 6)) #############################################################
        ################################################################## Again... Repeat using this

        print(f"\nFinal supercell vectors for {output_filename_alpha2} (Angstroms):")
        print(alpha2_supercell.get_cell())
        print(f"Total Atoms in supercell: {len(alpha2_supercell)}")

        write(output_filename_alpha2, alpha2_supercell, format="lammps-data")
        print(f"\nSuccessfully wrote LAMMPS data file to: {output_filename_alpha2}")
        preview_file_content(output_filename_alpha2)

    except Exception as e:
        print(f"\nAn error occurred during creation or writing for {output_filename_alpha2}: {e}")


if __name__ == "__main__":
    main()




--- Generating gamma_tial_oriented_4x4x4.data ---
Orientation: X=[1-10], Y=[11-2], Z=[111]

Original cell vectors (default [100], [010], [001]):
Cell([4.02, 4.02, 4.1004])

Oriented unit cell vectors (before replication):
Cell([[4.02, -4.02, 0.0], [4.02, 4.02, -8.2008], [4.02, 4.02, 4.1004]])
Atoms in unit cell: 24
Oriented cell PBC: [ True  True  True]

Final supercell vectors for gamma_tial_oriented_4x4x4.data (Angstroms):
Cell([[28.139999999999997, -28.139999999999997, 0.0], [16.08, 16.08, -32.8032], [16.08, 16.08, 16.4016]])
Total Atoms in supercell: 2688

Successfully wrote LAMMPS data file to: gamma_tial_oriented_4x4x4.data

--- File Content Preview: gamma_tial_oriented_4x4x4.data (first 15 lines) ---
(written by ASE)

2688 atoms
2 atom types

0.0      39.795969645178893  xlo xhi
0.0      39.914693162293005  ylo yhi
0.0      28.033396398453441  zlo zhi
0                       0    -0.52342041150241592  xy xz yz

Atoms # atomic

1   2      2.8425692603699204  5.2567905449918896e-